<a href="https://colab.research.google.com/github/mequanent/Social-Networks/blob/main/SN_Link_Prediction_Second_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**load graphs**

In [ ]:
#!pip install networkx
import networkx as nx
import time
import numpy as np
import pandas as pd

# load graph
G = nx.karate_club_graph()
# graph statistics
print(G.number_of_nodes())
print(G.number_of_edges())
print(list(G.nodes))
print(list(G.neighbors(0)))

34
78
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 17, 19, 21, 31]


Remove edges

In [ ]:
# remove edges
edge_to_remove = [75, 51, 18]
# remove edges
removed_edges = list()
for i in edge_to_remove:
    u = list(G.edges)[i][0]
    v = list(G.edges)[i][1]
    removed_edges.append([u,v])
    G.remove_edge(u,v)
k = len(removed_edges)
print(removed_edges)

[[31, 32], [18, 33], [1, 7]]


## LCC

In [ ]:
#local clustering coffecient
def local_clustering_coefficient(g):
  LCC=dict()
  for i in g.nodes:
    triangles=nx.triangles(g,i)
    k= nx.degree(g,i)
    lcc=0
    if k>1:
      lcc=2*triangles/(k*(k-1))
    LCC[i]=lcc

  lccs=[]
  for u in g.nodes:
    for v in g.nodes:
      if u not in nx.neighbors (g,v) and u<v:
        lccs.append((u,v,LCC[u]+LCC[v]))
  return lccs

## simRank

In [ ]:

from itertools import product

def sim(g, old_sim, u, v):
  s = list(product (g.adj[u], g.adj[v]))
  sim_sum = sum(old_sim[w][x] for (w, x) in s) / len(s) if s else 0.0
  return sim_sum

def simrank_similarity(g, C=0.8, K=5):
  newsim = {u: {v: 1.0 if u == v else 0.0 for v in g} for u in g}
  for its in range(K):
    old_sim = newsim.copy()
    newsim = {u: {v: C*sim(g, old_sim, u, v) if u is not v else 1 for v in g} for u in g}

  simrank_nodes = []
  for u, y in newsim.items () : 
    for v, val in y.items () :
      if u < v and u not in g.neighbors(v):
        simrank_nodes.append([u,v,val])
  return simrank_nodes 

In [ ]:
cn = nx.common_neighbor_centrality(G)
# sort Common Neighbors
sorted_cn = sorted(cn, key=lambda tup: tup[2], reverse=True)

#jaccard coefficient
jac = nx.jaccard_coefficient(G)
# sort jaccard coefficient
sorted_jac = sorted(jac, key=lambda tup: tup[2], reverse=True)

#adamic_adar_index 
adam = nx.adamic_adar_index(G)
# sort adam
sorted_adam = sorted(adam, key=lambda tup: tup[2], reverse=True)

#preferential_attachment
pre = nx.preferential_attachment(G)
# sort preferntial attachment
sorted_pre = sorted(pre, key=lambda tup: tup[2], reverse=True)

# local clustring coeffient 
lcc=local_clustering_coefficient(G)
#sort local clustring coeffient
sorted_lcc = sorted(lcc,key=lambda tup: tup[2],reverse=True)

#sort simRank similarity
sorted_simr = sorted(simrank_similarity(G), key=lambda tup: tup [2], reverse= True)

## Metrics 

In [ ]:
def metrics(cof, N, removed_edges):  
  Mean_rank = 0
  reciprocal_rank = 0
  idx = 0
  #N = 100
  hit = 0

  start = time.time()
  for u,v,val in cof:
      if [u,v] in removed_edges:
        Mean_rank += idx
        reciprocal_rank += 1/(idx+1)
        if idx <= 100:
          hit += 1
      idx += 1
  time_ = time.time() - start
  
  return Mean_rank/k, reciprocal_rank/k, hit, time_

## Display

In [ ]:
def show(coef):
  Mean_rank, reciprocal_rank, hit, time_ = metrics(coef, 100, removed_edges)
  print("MR", Mean_rank)
  print("MRR", reciprocal_rank)
  print("Hit@100", hit)
  print("Time：%f seconds" % time_)
  print()

In [ ]:
#sorted_cn, sorted_jac, sorted_adam, sorted_pre, sorted_lcc, sorted_simr 
print("Common Neighbor")
show(sorted_cn) 
print("Jaccard Coefficient")
show(sorted_jac) 
print("Adamic/Adar")
show(sorted_adam)
print("Preferential Attachment") 
show(sorted_pre) 
print("Local Clustering Coefficient")
show(sorted_lcc) 
print("simRank")
show(sorted_simr) 

Common Neighbor
MR 160.0
MRR 0.11390640802405506
Hit@100 1
Time：0.000288 seconds

Jaccard Coefficient
MR 184.33333333333334
MRR 0.009301753947423238
Hit@100 1
Time：0.000277 seconds

Adamic/Adar
MR 131.0
MRR 0.03173144573144573
Hit@100 1
Time：0.000255 seconds

Preferential Attachment
MR 74.0
MRR 0.029221522292215224
Hit@100 2
Time：0.000223 seconds

Local Clustering Coefficient
MR 370.3333333333333
MRR 0.0032556723687336662
Hit@100 0
Time：0.000249 seconds

simRank
MR 179.0
MRR 0.006456824828917852
Hit@100 1
Time：0.000189 seconds



# Homework

## metric function

In [ ]:
def get_metrics(cof, removed_edges):  
  MRR = 0
  MR = 0
  idx = 0
  hit50 = 0
  hit100 = 0
  hit300 = 0 

  start = time.time()
  for u,v,val in cof:
      if [u,v] in removed_edges:
        MR += idx
        MRR += 1/(idx+1)
        if idx <= 50:
          hit50 += 1
        if idx <= 100:
          hit100 += 1
        if idx <= 300:
          hit300 += 1
      idx += 1
  time_ = time.time() - start
  
  return MR/k, MRR/k, hit50, hit100, hit300, time_

In [ ]:
def remove_edges(g, edge_to_remove):
  # remove edges
  removed_edges = list()
  for i in edge_to_remove:
      u = list(g.edges)[i][0]
      v = list(g.edges)[i][1]
      removed_edges.append([u,v])
      g.remove_edge(u,v)
  k = len(removed_edges)
  
  return removed_edges 

In [ ]:
# get and descending sort the similarity coefficients   
def get_coefficients(g):
  # Common Neighbor
  cn = nx.common_neighbor_centrality(G)
  sorted_cn = sorted(cn, key=lambda tup: tup[2], reverse=True)

  # jaccard coefficient
  jac = nx.jaccard_coefficient(G)
  sorted_jac = sorted(jac, key=lambda tup: tup[2], reverse=True)

  #adamic_adar_index 
  adam = nx.adamic_adar_index(G)
  sorted_adam = sorted(adam, key=lambda tup: tup[2], reverse=True)

  #preferential_attachment
  pre = nx.preferential_attachment(G)
  sorted_pre = sorted(pre, key=lambda tup: tup[2], reverse=True)

  #local clustring coeffient 
  lcc=local_clustering_coefficient(G)
  sorted_lcc = sorted(lcc,key=lambda tup: tup[2],reverse=True)

  #simRank similarity
  sorted_simr = sorted(simrank_similarity(G), key=lambda tup: tup [2], reverse= True)

  return sorted_cn, sorted_jac, sorted_adam, sorted_pre, sorted_lcc, sorted_simr

## 10 iterations

In [ ]:
columns = ['MR', 'MRR', 'Hit@50', 'Hit@100', 'Hit@300', 'time']
indices = ['CN', 'Jaccard', 'Preferential Attachment', 'Adamic', 'LCC', 'simRank']

df = pd.DataFrame(0.0, index=indices, columns=columns)
df

,MR,MRR,Hit@50,Hit@100,Hit@300,time
CN,0.0,0.0,0.0,0.0,0.0,0.0
Jaccard,0.0,0.0,0.0,0.0,0.0,0.0
Preferential Attachment,0.0,0.0,0.0,0.0,0.0,0.0
Adamic,0.0,0.0,0.0,0.0,0.0,0.0
LCC,0.0,0.0,0.0,0.0,0.0,0.0
simRank,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def one_K_metrics(k):
  columns = ['MR', 'MRR', 'Hit@50', 'Hit@100', 'Hit@300', 'time']
  indices = ['CN', 'Jaccard', 'Preferential Attachment', 'Adamic', 'LCC', 'simRank']

  df = pd.DataFrame(0.0, index=indices, columns=columns)
  for i in range(10): 
    N = 100
    g = nx.karate_club_graph() 
    edges_to_remove = np.random.choice(range(len(g.edges)), k, replace=False)
    edges_to_remove = sorted(edges_to_remove, reverse=True)
    removed_edges = remove_edges(g, edges_to_remove)

    cn, jac, adam, pre, lcc, simr = get_coefficients(G)
    #coefs = [get_coefficients(G)]

    coefficients = {'CN': cn, 'Jaccard': jac, 
                      'Preferential Attachment': pre, 
                      'Adamic': adam, 'LCC': lcc, 'simRank': simr}
  #coef_metric = dict()   

    MR, MRR, hit50, hit100, hit300, time_ = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    for i in coefficients.keys():
      # get MR, MRR, hit50, hit100, hit300, time_
      #MR, MRR, hit50, hit100, hit300, time_ += [metrics(i, removed_edges)] 
      df.loc[i, :] += get_metrics(coefficients[i], removed_edges)  
  df /= 10

  return df


In [ ]:
# do for all Ks
K = [1, 5, 10, 50]

for k in K:
  print(f"The corresponding metrics for K = {k}")
  print(one_K_metrics(k).to_markdown())
  print()

The corresponding metrics for K = 1
|                         |   MR |         MRR |   Hit@50 |   Hit@100 |   Hit@300 |        time |
|:------------------------|-----:|------------:|---------:|----------:|----------:|------------:|
| CN                      | 25.8 | 0.0003861   |      0   |       0   |       0.1 | 0.000109243 |
| Jaccard                 | 25.1 | 0.000396825 |      0   |       0   |       0.1 | 0.000120401 |
| Preferential Attachment |  1.4 | 0.00666667  |      0.1 |       0.1 |       0.1 | 0.000101209 |
| Adamic                  | 25.8 | 0.0003861   |      0   |       0   |       0.1 | 0.000111079 |
| LCC                     | 44.7 | 0.000223214 |      0   |       0   |       0   | 0.000117397 |
| simRank                 | 25.7 | 0.000387597 |      0   |       0   |       0.1 | 0.000138402 |

The corresponding metrics for K = 5
|                         |    MR |         MRR |   Hit@50 |   Hit@100 |   Hit@300 |        time |
|:------------------------|------:|---------